## Creating Toll Attribution Table by Merging subscription data and tolls data 

Only need to execute once

In [ ]:
CREATE TABLE "Wabash_tolls"."toll_attribution" AS
SELECT 
    t."Vehicle_Number",
    t."Exit_Date",
    t."Toll_Charge",
    s."Client",
    s."Start datetime",
    s."End datetime"
FROM "Wabash_tolls"."tolls" t
JOIN "Wabash_tolls"."subscriptions" s
ON TRIM(t."Vehicle_Number") = TRIM(s."Asset name")
AND DATE(t."Exit_Date") BETWEEN DATE(s."Start datetime") AND DATE(s."End datetime");


## Updating all the new matchings to the toll attrinution table

execute repeatedly

In [ ]:
INSERT INTO Wabash_tolls.toll_attribution
SELECT 
    t.Vehicle_Number,
    t.Exit_Date,
    t.Toll_Charge,
    s.Client,
    s."Start datetime",
    s."End datetime",
    CURRENT_TIMESTAMP AS update_timestamp
FROM Wabash_tolls.tolls t
JOIN Wabash_tolls.subscriptions s
ON TRIM(t.Vehicle_Number) = TRIM(s."Asset name")
AND t.Exit_Date BETWEEN s."Start datetime" AND s."End datetime"
WHERE NOT EXISTS (
    SELECT 1 
    FROM Wabash_tolls.toll_attribution a
    WHERE a.Vehicle_Number = t.Vehicle_Number
    AND a.Exit_Date = t.Exit_Date
);


## Write a query that shows the total toll amount by date and clients

In [ ]:
SELECT 
    DATE(Exit_Date) AS Toll_Date,
    Client,
    SUM(Toll_Charge) AS Total_Toll_Charge,
    COUNT(DISTINCT Vehicle_Number) AS Vehicles_Charged
FROM Wabash_tolls.toll_attribution
GROUP BY 
    DATE(Exit_Date), 
    Client
ORDER BY 
    Toll_Date DESC, 
    Client;
